# simple example - linear logic

This example shows how to speed up a calculation using multi processing and caching via `mppfc` module.

The logic here is linear.
First, do the calculation in paralell while disregarding the results.
After the calculation has finished, calling the calculation a second time will get the results from cache.

This means that there is some boilerplate code by splitting the whole procedure into "calculation" and "data collection" parts.

In [ ]:
import mppfc
import time

# number of function calls, each call takes about 2 seconds
N = 10

### straight forward implementaion

In [ ]:
def slow_function(x):
    """some important calculation"""
    time.sleep(2)
    return x

In [ ]:
t0 = time.perf_counter_ns()

data = []
for x in range(N):
    y = slow_function(x)
    data.append([x, y])

time_basic = (time.perf_counter_ns() - t0) / 10**9
print("straight forward implementation with {} data points takes {:.1f} seconds".format(len(data), time_basic))

### same code with mppfc decorator

In [ ]:
@mppfc.MultiProcCachedFunctionDec()
def slow_function(x):
    """some important calculation"""
    time.sleep(2)
    return x

### split routine in "calculation"

We start the multiprocessing mode with `start_mp()` and simply call our function `slow_function` without caring about its results.By calling `wait()`, we wait until all parameters have been processed and cached to disk.

In [ ]:
# remove cache from possible former calculation for the sakes of the example
import shutil
shutil.rmtree(slow_function.cache_dir)

t0 = time.perf_counter_ns()

# Here we use 2 subprocesses.
# You can also use 'all' to use as many processes as cores available
# or specify a portion of them by passing a float within the interval (0.0, 1.0].
# A negative int specifies the number of cores NOT to use.
slow_function.start_mp(num_proc=2)

# now, calling slow_function will pass the argument to a subprocesses and return immediately.
t1 = time.perf_counter_ns()
for x in range(N):
    slow_function(x)
t2 = time.perf_counter_ns()
print("calling the function returns nearly immediately, {:.3g}s".format((t2-t1) / 10**9))

# wait until all arguments have been processed
# show the status every second
slow_function.wait(status_interval_in_sec=1)   

t3 = time.perf_counter_ns()
time_mppfc = (time.perf_counter_ns() - t0) / 10**9
print("mp accelerated calculation takes {:.3g} seconds".format(time_mppfc))

### ... and "data collection"

Now we can call `slow_function` as usual and use its return value.
Adding the `_cache_flag="cache_only"` parameter is not necessary.
However, it emphasizes that all results are taken from the cache.

In [ ]:
t0 = time.perf_counter_ns()

data = []
for x in range(N):
    y = slow_function(x)
    data.append([x, y])

time_cache = (time.perf_counter_ns() - t0) / 10**9
print("fill data from cache takes {:.3g} seconds".format(time_cache))